## NLP assignment

In [ ]:
import os
import spacy
import PyPDF2
import re
import random

# Load English language model
nlp = spacy.load("en_core_web_sm")

...

def generate_mcq_with_multiple_correct(question, correct_answers, other_options, num_options=4):
  options = correct_answers + other_options
  random.shuffle(options)

  mcq = {
    "question": question,
    "options": options[:num_options],
    "correct_answers": correct_answers
  }

  return mcq

def generate_variety_question(doc):
  sentence = random.choice(list(doc.sents))
  blank_word = random.choice([token for token in sentence if not token.is_punct])

  question_text = sentence.text.replace(blank_word.text, "______")
  correct_answers = [blank_word.text]

  other_options = [token.text for token in doc if token.is_alpha and token.text != correct_answers[0]]
  num_correct_options = random.randint(1, 2) # Generate 1 or 2 correct options
  correct_answers.extend(random.sample(other_options, num_correct_options))

  num_other_options = min(4 - num_correct_options, len(other_options))
  other_options = random.sample(other_options, num_other_options)

  mcq = generate_mcq_with_multiple_correct(question_text, correct_answers, other_options, num_options=4)
  return mcq

def get_mca_questions(context: str, num_questions: int):
  doc = nlp(context)

  questions = [generate_variety_question(doc) for _ in range(num_questions)]

  mca_questions = []
  for i, question in enumerate(questions, start=1):
    question_str = f"Q{i}: {question['question']}\n"
    options_str = ""
    for j, option in enumerate(question['options']):
      options_str += f"{j + 1}. {option}\n"

    correct_options_formatted = " & ".join([f"({chr(ord('a') + question['options'].index(ans))})" if ans in question['options'] else f"({ans})" for ans in question['correct_answers']])

    mca_question = f"{question_str}{options_str}\nCorrect Options : {correct_options_formatted}\n"
    mca_questions.append(mca_question)

  return mca_questions

...



if __name__ == "__main__":
    folder_path = "/content/Dataset"
    num_questions_per_file = 5  # Number of questions to generate per PDF file

    mca_questions = process_pdf_folder(folder_path, num_questions_per_file)

    for i, question in enumerate(mca_questions, start=1):
        print(question)